In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
from exp.nb_07 import *

## Layerwise Sequenctial Unit Variance (LSUV)
Getting the MNIST data and a CNN

In [3]:
x_train, y_train, x_valid, y_valid = get_mnist_data()

x_train, x_valid = normalize_to(x_train, x_valid)
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

nh, bs = 50, 512
c = y_train.max().item() + 1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
mnist_view = view_tfm(1, 28, 28)

device = torch.device('cuda', 0) if torch.cuda.is_available else torch.device('cpu')
print(device)

cbfs = [Recorder,
        partial(AvgStatsCallback, accuracy),
        partial(CudaCallback, device),
        partial(BatchTransformXCallback, mnist_view)]

cuda:0


In [5]:
nfs = [8, 16, 32, 64, 64]

In [6]:
class ConvLayer(nn.Module):
  def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
    super().__init__()
    self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
    self.relu = GeneralReLU(sub=sub, **kwargs)
    
  def forward(self, x):
    return self.relu(self.conv(x))
  
  @property
  def bias(self):
    return - self.relu.sub
  
  @bias.setter
  def bias(self, v):
    self.relu.sub = -v
    
  @property
  def weight(self):
    return self.conv.weight

In [7]:
learn, run = get_learn_run(nfs, data, lr=0.6, layer=ConvLayer, cbs=cbfs)

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [8]:
run.fit(2, learn)

train: [2.30139375, tensor(0.1652, device='cuda:0')]
valid: [1.6644787109375, tensor(0.3890, device='cuda:0')]
train: [1.5066678125, tensor(0.4475, device='cuda:0')]
valid: [0.44689287109375, tensor(0.8668, device='cuda:0')]


Now we recreate our model and we'll try again with LSUV. Hopefully, we'll get better results!

In [9]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

Helper function to get one batch of a given dataloader, with the callbacks called to preprocess it.

In [10]:
#export
def get_batch(dl, run):
  run.xb, run.yb = next(iter(dl))
  for cb in run.cbs:
    cb.set_runner(run)
  run('begin_batch')
  return run.xb, run.yb

In [11]:
xb, yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [12]:
#export
def find_modules(m, cond):
  if cond(m):
    return [m]
  return sum([find_modules(o, cond) for o in m.children()], [])

def is_lin_layer(l):
  lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)
  return isinstance(l, lin_layers)

In [13]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralReLU()
 ),
 ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralReLU()
 ),
 ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralReLU()
 ),
 ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralReLU()
 ),
 ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralReLU()
 )]

This is a helper function to grab the mean and std of the output of a hooked layer.

In [14]:
def append_stat(hook, mod, inp, outp):
  d = outp.data
  hook.mean = d.mean().item()
  hook.std = d.std().item()

In [15]:
device = torch.device('cuda', 0) if torch.cuda.is_available else torch.device('cpu')
mdl = learn.model.to(device)

So now we can look at the mean and std of the conv layers of our model.

In [16]:
with Hooks(mods, append_stat) as hooks:
  mdl(xb)
  for hook in hooks:
    print(hook.mean, hook.std)

0.19760100543498993 0.32771408557891846
0.07450263202190399 0.12166350334882736
0.03472357243299484 0.0550089068710804
0.024658890441060066 0.03108115680515766
0.010263403877615929 0.014492860995233059


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated.

In [17]:
#export
def lsuv_module(m, xb):
  h = Hook(m, append_stat)
  
  while mdl(xb) is not None and abs(h.mean) > 1e-3:
    m.bias -= h.mean
  while mdl(xb) is not None and abs(h.std - 1) > 1e-3:
    m.weight.data /= h.std
    
  h.remove()
  return h.mean, h.std

In [18]:
for m in mods:
  print(lsuv_module(m, xb))

(0.3027128577232361, 0.999974250793457)
(0.3601396977901459, 1.0002014636993408)
(0.3490963876247406, 0.9999998807907104)
(0.4014589190483093, 0.999965488910675)
(0.5185941457748413, 0.9995793104171753)


Note that, the mean doesn't exactly stay at 0, since we change the standard deviation after by scaling the weight.
Then training is beginning on better grounds.

In [19]:
%time run.fit(2, learn)

train: [0.679314609375, tensor(0.7858, device='cuda:0')]
valid: [0.145125341796875, tensor(0.9569, device='cuda:0')]
train: [0.1206329296875, tensor(0.9626, device='cuda:0')]
valid: [0.09196317138671875, tensor(0.9733, device='cuda:0')]
CPU times: user 3.49 s, sys: 3.45 ms, total: 3.49 s
Wall time: 3.47 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [20]:
!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to nb_07a.py
